This is one of the Objectiv example notebooks. For more examples visit the 
[example notebooks](https://objectiv.io/docs/modeling/example-notebooks/) section of our docs. The notebooks can run with the demo data set that comes with the our [quickstart](https://objectiv.io/docs/home/try-the-demo/), but can be used to run on your own collected data as well.

All example notebooks are also available in our [quickstart](https://objectiv.io/docs/home/try-the-demo/). With the quickstart you can spin up a fully functional Objectiv demo pipeline in five minutes. This also allows you to run these notebooks and experiment with them on a demo data set.

# Basic user intent analysis

In this notebook, we briefly demonstrate how you can easily do basic user intent analysis on your data.

## Getting started
If you are running this example on your own collected data, [see the instructions here](https://objectiv.io/docs/modeling/get-started-in-your-notebook/) on how to setup the database connection and get started in your favorite notebook tool.

### Import the required packages for this notebook
The open model hub package can be installed with `pip install objectiv-modelhub` (this installs Bach as well).  
If you are running this notebook from our quickstart, the model hub and Bach are already installed, so you don't have to install it separately.

In [ ]:
from modelhub import ModelHub, display_sql_as_markdown
import bach
import pandas as pd
from datetime import timedelta

We first have to instantiate the model hub and an Objectiv DataFrame object.

In [ ]:
# instantiate the model hub, set the default time aggregation to daily and get the application global context
modelhub = ModelHub(time_aggregation='%Y-%m-%d', global_contexts=['application'])
# get the Bach DataFrame with Objectiv data
df = modelhub.get_objectiv_dataframe(start_date='2022-02-01', end_date='2022-05-01')

The `location_stack` column, and the columns taken from the global contexts, contain most of the event-specific data. These columns are JSON typed, and we can extract data from it using the keys of the JSON objects with [`SeriesLocationStack`](https://objectiv.io/docs/modeling/open-model-hub/api-reference/SeriesLocationStack/SeriesLocationStack/) methods, or the `context` accessor for global context columns. See the [open taxonomy example](open-taxonomy-how-to.ipynb#Location-stack-&-global-contexts) for how to use the `location_stack` and global contexts. 

In [ ]:
# adding specific contexts to the data as columns
df['application_id'] = df.application.context.id
df['root_location'] = df.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')

## Exploring root location
The `root_location` context in the `location_stack` uniquely represents the top-level UI location of the user. As a first step of grasping user intent, this is a good starting point to see in what main areas of your product users are spending time.

In [ ]:
# model hub: unique users per application and root location
users_root = modelhub.aggregate.unique_users(df, groupby=['application_id', 'root_location'])
users_root.sort_index().head(10)

## Exploring session duration
The average `session_duration` model from the [open model hub](https://objectiv.io/docs/modeling/open-model-hub/) is
another good pointer to explore first for user intent.

In [ ]:
# model hub: duration, per application and root location
duration_root = modelhub.aggregate.session_duration(df, groupby=['application_id', 'root_location']).sort_index()
duration_root.head(10)

Now, we can look at the distribution of time spent. We used the Bach `quantile` operation for this. We'll use this distribution to define the different stages of user intent.  

In [ ]:
# how is this time spent distributed?
session_duration = modelhub.aggregate.session_duration(df, groupby='session_id')
# materialization is needed because the expression of the created series contains aggregated data, 
# and it is not allowed to aggregate that.
session_duration = session_duration.materialize()

# show quantiles
session_duration.quantile(q=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).head(10)

## Defining different stages of user intent
After exploring the `root_location` and `session_duration` (both per root location and quantiles), we can make a simple definition of different stages of user intent.

Based on the objectiv.io website data in the quickstart:

We think that users that spent most time (90th percentile) and specifically in our documentation sections are in the Implement phase of Objectiv. As there is a jump beyond the one minute mark at the 70th percentile, it feels sensible to deem that users beyond the 70th up to 90th perctile in our documentation sections are Exploring. The remaining users are Informing themselves about the product. Those users are spending less than 1:40 in the docs and/or spend any amount of time on our main website.

| User intent | Root locations | Duration |
| :--- | :--- | :--- |
| 1 - Inform | *all sections other than the ones mentioned below* | *any time spent* |
| 1 - Inform | Docs: modeling, taxonomy, tracking, home | less than 1:40 |
| 2 - Explore | Docs: modeling, taxonomy, tracking, home | between 1:40 and 11:30 |
| 3 - Implement | Docs: modeling, taxonomy, tracking, home | more than 11:30 | 

This is just for illustration purposes, you can adjust these definitions based on your own collected data. 

## Assigning user intent
Based on the definitions above, we can start assigning a stage of intent to each user. 

In [ ]:
# set the root locations that we will use based on the definitions above
roots = bach.DataFrame.from_pandas(engine=df.engine, 
                                   df=pd.DataFrame({'roots': ['modeling', 'taxonomy', 'tracking', 'home', 'docs']}), 
                                   convert_objects=True).roots

In [ ]:
# now we calculate the total time spent per _user_ and create a data frame from it
user_intent_buckets = modelhub.agg.session_duration(df, 
                                                    groupby=['user_id'], 
                                                    method='sum',
                                                    exclude_bounces=False).to_frame()

In [ ]:
# same as above, but for selected roots only
selector = (df.root_location.isin(roots)) & (df.application_id=='objectiv-docs')
explore_inform_users_session_duration = modelhub.agg.session_duration(df[selector], groupby='user_id', method='sum')
# and set it as column
user_intent_buckets['explore_inform_duration'] = explore_inform_users_session_duration

In [ ]:
# first, we set the Inform bucket as a catch-all, meaning users that do not fall into Explore and Implement will be defined as Inform
user_intent_buckets['bucket'] = '1 - inform'

In [ ]:
# calculate buckets duration
user_intent_buckets.loc[(user_intent_buckets.explore_inform_duration >= timedelta(0, 100)) &
                        (user_intent_buckets.explore_inform_duration <= timedelta(0, 690)), 'bucket'] = '2 - explore'

user_intent_buckets.loc[user_intent_buckets.explore_inform_duration > timedelta(0, 690), 'bucket'] = '3 - implement'

Now, we have assigned intent to each user and can for example look at the total number of users per intent bucket.

In [ ]:
# total number of users per intent bucket
user_intent_buckets.reset_index().groupby('bucket').agg({'user_id': 'nunique'}).sort_index().head()

## What's next?

The are many next possible analysis steps, for example:
- What product features do each of the intent groups use? 
- What kind of intent users come from different marketing campaigns? 
- How can we drive more users to the 'Implement' stage? Look at different product features that users with the 'Implement' intent use, compared to 'Explore'.

A good starting point for these analyses on top of the user intent buckets is the basic product analytics example in the [example notebooks](https://objectiv.io/docs/modeling/example-notebooks/).

## Get the SQL for this user intent analysis
The SQL for any analysis can be exported with one command, so you can use models in production directly to simplify data debugging & delivery to BI tools like Metabase, dbt, etc. See how you can [quickly create BI dashboards with this](https://objectiv.io/docs/home/try-the-demo#creating-bi-dashboards).

In [ ]:
# get the SQL to use this analysis in for example your BI tooling
display_sql_as_markdown(user_intent_buckets)